In [1]:
# USEFUL WHEN RUNNING ON CLUSTER
import sys
!pip install torch torchvision torchsummary torchtext pytorch_lightning tensorboard matplotlib tqdm datetime time 

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [2]:
!nvidia-smi

Fri Jan 13 10:39:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 455.23.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:86:00.0 Off |                  Off |
| N/A   33C    P0    24W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [34]:
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.nn import BCELoss
from torch.nn import MSELoss
from torch.nn import functional as F

from torch.optim import Adam
from torch.optim import SGD
from torch.optim.lr_scheduler import CyclicLR
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import datetime

from src.model import UNet
from src.dataloader import LandCoverData
import src.loss as lossPY

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
#path="../"
path="/scratch/izar/damiani/"
#path="/scratch/izar/nkaltenr/"

train_dataset = LandCoverData(path, split="train", ignore_last_number=11, use_augmented=True)
val_dataset = LandCoverData(path, split="val")

In [52]:
BATCH_SIZE = 16

# num_workers 8 default but 2 on colab
train_dl = DataLoader(train_dataset, BATCH_SIZE, True, drop_last=True)
val_dl = DataLoader(val_dataset, BATCH_SIZE, False, drop_last=True)

In [53]:
INPUT_IMAGE_HEIGHT=200
INPUT_IMAGE_WIDTH=200

DEVICE = "cuda"
#DEVICE = "cpu"
INIT_LR = 1e-3
INIT_MOMENTUM = 0.9
INIT_WEIGHT_DECAY = 0.01
NUM_EPOCHS = 400

In [54]:
focal_loss = torch.hub.load(
	'adeelh/pytorch-multi-class-focal-loss',
	model='focal_loss',
	alpha=None,
	gamma=2,
	reduction='mean',
	device=DEVICE,
	dtype=torch.float32,
	force_reload=False
)

Using cache found in /home/damiani/.cache/torch/hub/adeelh_pytorch-multi-class-focal-loss_master


In [75]:
#weight_ce = [1., 3., 3.1, 14., 7., 3.3, 0.9, 1.]
#class_weights_ce = torch.FloatTensor(weight_ce).to(DEVICE)
#ce = CrossEntropyLoss(weight=class_weights_ce)
ce = CrossEntropyLoss()

#INIT_ALPHA = 10.0
#INIT_GAMMA = 2.0
#ml = MixedLoss(INIT_ALPHA, INIT_GAMMA)
#ftl = FocalTverskyLoss()
iou=lossPY.mIoULoss(n_classes=8).to(DEVICE)


def UnetLoss(preds, targets):
    #print(f"preds : {preds.shape}")
    #print(f"targets : {targets.shape}")
    #ce_loss = ce(preds, targets)
    #d_loss = dice_coef_8cat_loss(targets, preds)
    
    iou_loss = iou(preds, targets)
    
    #mix_loss = ce_loss + d_loss
    #ml_loss = ml
    #fl = focal_loss(preds, targets)
    #ftl_loss = ftl(preds, targets)
    acc = (torch.max(preds, 1)[1] == targets).float().mean()
    return iou_loss, acc

In [76]:
from torch.optim import lr_scheduler

In [77]:
# initialize our UNet model
unet = UNet(nbClasses=8
           ).to(DEVICE)
# initialize loss function and optimizer
lossFunc = UnetLoss
#opt = SGD(unet.parameters(), lr=INIT_LR, momentum=INIT_MOMENTUM)
opt = Adam(unet.parameters(), lr=0.001)
#scheduler = lr_scheduler.StepLR(opt, step_size=5, gamma=0.5)
#opt = Adam(unet.parameters(), lr=INIT_LR, weight_decay=INIT_WEIGHT_DECAY)
# calculate steps per epoch for training and test set
trainSteps = len(train_dataset) // BATCH_SIZE
testSteps = len(val_dataset) // BATCH_SIZE
# initialize a dictionary to store training history
H = {"train_loss": [], "test_loss": [], "train_acc": [], "test_acc": []}
bestValLoss = float('inf')
bestValAcc = -1

In [78]:
#H = torch.load('unet_model_2023-01-03_0:54_fl_2_loss_dict.pth', map_location=torch.device(DEVICE))

In [79]:
#unet.load_state_dict(torch.load('unet_model_epoch_200_fl_2_loss.pth', map_location=torch.device(DEVICE)))

In [80]:
# loop over epochs
NUM_EPOCHS = 100
print("[INFO] training the network...")
startTime = time.time()
#for e in tqdm(range(NUM_EPOCHS)):
for e in tqdm(range(NUM_EPOCHS)):
    # set the model in training mode
    unet.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalTestLoss = 0
    totalTrainAcc = 0
    totalTestAcc = 0
    # loop over the training set
    for (i, (x, y)) in enumerate(train_dl):
        # send the input to the device
        (x, y) = (x.to(DEVICE), y.to(DEVICE))
        # perform a forward pass and calculate the training loss
        pred = unet(x)
        y = y.to(torch.long)
        y = y.squeeze()
        
        loss, acc = lossFunc(pred, y)
        
        # first, zero out any previously accumulated gradients, then
        # perform backpropagation, and then update model parameters
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far
        totalTrainLoss += loss
        totalTrainAcc += acc
    # switch off autograd
    with torch.no_grad():
        # set the model in evaluation mode
        unet.eval()
        # loop over the validation set
        for (x, y) in val_dl:
            # send the input to the device
            (x, y) = (x.to(DEVICE), y.to(DEVICE))
            # make the predictions and calculate the validation loss
            pred = unet(x)
            #pred=pred.to(torch.float32)
            y = y.to(torch.long)
            y = y.squeeze()
            
            loss, acc = lossFunc(pred, y)
            totalTestLoss += loss
            totalTestAcc += acc
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgTestLoss = totalTestLoss / testSteps
    avgTrainAcc = totalTrainAcc / trainSteps
    avgTestAcc = totalTestAcc / testSteps
    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["test_loss"].append(avgTestLoss.cpu().detach().numpy())
    H["train_acc"].append(avgTrainAcc.cpu().detach().numpy())
    H["test_acc"].append(avgTestAcc.cpu().detach().numpy())
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, NUM_EPOCHS))
    print("       train loss: {:.4f}, val loss: {:.4f}".format(
      avgTrainLoss, avgTestLoss))
    print("       train acc: {:.4f}%, val acc: {:.4f}%".format(
      avgTrainAcc, avgTestAcc))
    # Save the best model (the one that has the lowest loss for validation)
    if (bestValLoss == -1) or (bestValLoss > avgTestLoss):
        bestValLoss = avgTestLoss
        print("best loss => saving")
        torch.save(unet.state_dict(), 'best_model_fl_2_loss.pth')
    if (bestValAcc < avgTestAcc):
        bestValAcc = avgTestAcc
        print("best acc => saving")
        torch.save(unet.state_dict(), 'best_model_acc_fl_2_loss.pth')
    if ((e+1)%50 == 0):
        epoch_name = 200 + e+1
        print("SAVING")
        torch.save(unet.state_dict(), f"unet_model_epoch_{epoch_name}_fl_2_loss.pth")
        
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

[INFO] training the network...


  0%|                                                                                               | 0/100 [00:00<?, ?it/s]

[INFO] EPOCH: 1/100
       train loss: 0.8808, val loss: 0.8956
       train acc: 0.5893%, val acc: 0.5467%
best loss => saving
best acc => saving


  1%|▊                                                                                   | 1/100 [02:13<3:40:48, 133.82s/it]

[INFO] EPOCH: 2/100
       train loss: 0.8717, val loss: 0.8961
       train acc: 0.6174%, val acc: 0.5499%
best acc => saving


...

[INFO] EPOCH: 98/100
       train loss: 0.7893, val loss: 0.8676
       train acc: 0.8310%, val acc: 0.6702%


 99%|██████████████████████████████████████████████████████████████████████████████████▏| 99/100 [4:21:10<02:51, 171.50s/it]

[INFO] EPOCH: 99/100
       train loss: 0.7887, val loss: 0.8790
       train acc: 0.8318%, val acc: 0.6034%
[INFO] EPOCH: 100/100
       train loss: 0.7880, val loss: 0.8765
       train acc: 0.8337%, val acc: 0.6206%
SAVING


100%|██████████████████████████████████████████████████████████████████████████████████| 100/100 [4:24:36<00:00, 158.77s/it]

[INFO] total time taken to train the model: 15876.52s


In [ ]:
date = datetime.datetime.now()
date_ymd = date.date()
date_hm = f"{date.hour}:{date.minute}"

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["test_loss"], label="val_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(f"train_val_loss_{date_ymd}_{date_hm}_fl_2_loss.png", bbox_inches='tight')

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["test_acc"], label="val_acc")
plt.title("Training Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig(f"train_val_acc_{date_ymd}_{date_hm}_fl_2_loss.png", bbox_inches='tight')

In [ ]:
torch.save(unet.state_dict(), f"unet_model_{date_ymd}_{date_hm}_fl_2_loss.pth")
torch.save(unet, f"unet_model_{date_ymd}_{date_hm}_fl_2_loss.pt")
torch.save(H, f"unet_model_{date_ymd}_{date_hm}_fl_2_loss_dict.pth")